In [112]:
import os
import copy
import glob
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
from matplotlib import colors
import numpy as np
import matplotlib.pyplot as plt
import netCDF4

In [55]:
resultdir = "../product/netCDF/1-UniformL/"
figdir1 = "../product/figures/1-UniformL"
os.makedirs(figdir) if not os.path.exists(figdir) else " "
resultlist = sorted(glob.glob(os.path.join(resultdir, "*.nc")))
domain = [-16., 9., 45., 66.]

### Projection

In [38]:
m = Basemap(llcrnrlon=domain[0], llcrnrlat=domain[2],
            urcrnrlon=domain[1], urcrnrlat=domain[3], 
            resolution = 'h', epsg=3035, projection='laea')

### EMODnet logo

In [54]:
logo = plt.imread("../data/logo_emodnet.png")

## Reading/plotting functions

In [107]:
def read_density(resultfile):
    with netCDF4.Dataset(resultfile) as nc:
        lon = nc.variables["lon"][:]
        lat = nc.variables["lat"][:]
        f = nc.variables["heatmap"][:]
        e = nc.variables["heatmap_error"][:]
        
    return lon, lat, f, e

In [193]:
def make_plot_field(lon, lat, density, figtitle=None, figname=None, error=None):
       
    # If the error field is defined, then it is used to mask the interpolated field
    if error is not None:
        print(error)
        density = (1. - error) * density;
        
    fig = plt.figure(figsize=(10, 10))
    m.drawmapboundary(fill_color='#F9F9F9', zorder=1)
    m.fillcontinents(color='#363636', zorder=4)
    m.drawparallels(np.arange(domain[2], domain[3], 4.), labels=[1,0,0,0], color="w", 
                    fontsize=16, zorder=3, linewidth=2, dashes=[2,1])
    m.drawmeridians(np.arange(domain[0], domain[1], 8.), labels=[0,0,0,1], color="w", 
                    fontsize=16, zorder=3, linewidth=2, dashes=[2,1])
    m.drawcountries(color="w", zorder=4)
    pcm = m.pcolormesh(lon, lat, density, latlon=True, cmap=cmap, shading="gouraud", 
                       vmin=0, vmax=1., zorder=2)
    plt.colorbar(pcm, shrink=.8)
    
    if figtitle is not None:
        plt.title(figtitle, fontsize=20)

    newax = fig.add_axes([0.625, 0.2, 0.125, 0.125], zorder=7)
    newax.imshow(logo)
    newax.axis('off')
    
    if figname is not None:
        plt.savefig(figname, dpi=300, bbox_inches="tight")
    else:
        plt.show()
    plt.close()

#### Colormap

In [194]:
hot_r = plt.cm.get_cmap('hot_r', 256)
newcolors = hot_r(np.linspace(0, 1, 256))
newcolors[0,0:3] = colors.hex2color("#F9F9F9")
cmap = colors.ListedColormap(newcolors)

## Create plots
### Lood coordinates from first file

In [195]:
lon, lat, field, error = read_density(resultlist[0])
llon, llat = np.meshgrid(lon, lat)

### Loop on the files

In [198]:
for resultfile in resultlist:
    
    # Read results
    _, _, field, error = read_density(resultfile)
    r = os.path.basename(resultfile)
    
    # Prepare figure name and title
    fname1 = r.replace(".nc", ".jpg")
    fname2 =  r.replace(".nc", "_masked.jpg")
    figtitle = r.replace("_density.nc", "").replace("_", " ")
    figname1 = os.path.join(figdir, fname1)
    figname2 = os.path.join(figdir, fname2)
    
    # Plot 
    # - Unmasked field
    make_plot_field(llon, llat, field, figtitle, figname1)
    # - masked field
    make_plot_field(llon, llat, field, figtitle, figname2, error)

[[1.         1.         1.         ...        nan        nan        nan]
 [1.         0.99999999 0.99999999 ...        nan        nan        nan]
 [0.99999999 0.99999999 0.99999999 ...        nan        nan        nan]
 ...
 [       nan        nan        nan ... 1.         1.         1.        ]
 [       nan        nan        nan ... 1.         1.         1.        ]
 [       nan        nan        nan ... 1.         1.         1.        ]]
